In [ ]:
#Nous pouvons passer à l'étape suivante

#PARTIE 3/ REGRESSION LOGISTIQUE

In [ ]:
#Pour faire le lien entre la partie 2 et 3
df=data

In [ ]:
# On définit les variables explicatives et les variables expliquées du modèle
features=['Age', 'Fare','SibSp']
X = df[features]  # On prend, de manière arbitraire, les variables 'Age', 'Fare' et 'SibSp' pour une première modélisation
y = df['Survived']

# On sépare les données, pour créer et tester le modèle (on ne peut pas faire les deux sur les mêmes données, car les estimateurs n'auraient pas de validité externe)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Spécifions le modèle
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

# Créons le vecteur des valeurs prédites
y_pred = model.predict(X_test)

# Calculons la précision du modèle
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
#Dans ce premier modèle, nous avons spécifié les variables explicatives de manière arbitraire.
#Maintenant, nous allons déterminer les variables explicatives selon un critère déterminé: la corrélation avec la variable expliquée.
#Nous allons reprendre le vecteur de corrélation vu précédement, et en extraire les variables le plus liées à la survie des passagers.
#Au final, on intègre dans notre régression les variables à coefficient de correlation avec 'Survived' >=0.3

In [ ]:
#Regarder la correlation des variables avec 'Survived'

numerical_features = ['Sex','Age', 'Fare', 'Parch', 'SibSp','Pclass','Embarked']

corr_matrices = {}

# Calculons le vecteur de corrélation
for feature in numerical_features:
    corr_matrix = df[[feature, 'Survived']].corr(method='spearman')  # Spearman for mixed data types
    corr_matrices[feature] = corr_matrix.iloc[1, 0]  # Extract correlation value with 'Survived'

# Affichons le vecteur de corrélation
print("Correlations avec 'Survived':")
for feature, correlation in corr_matrices.items():
    print(f"{feature}: {correlation:.2f}")


In [ ]:
# On extrait les variables les plus correllées
high_corr_features = []
for feature, correlation in corr_matrices.items():
    if abs(correlation) > 0.3:
        high_corr_features.append(feature)

# On affiche ces variables (si des variables à rho >= 0.3 existent)
if high_corr_features:
    print("Variables à coeff. de corrélation > 0.3 (en VA):")
    for feature in high_corr_features:
        print(f"- {feature}")
else:
    print("Il n'y a pas de variables à coeff. de corrélation > 0.3 (en VA) ")

In [ ]:
# On spécifie maintenant une régression logistique avec nos nouvelles variables.

#On affecte les variables à X
X = df[high_corr_features]
y = df['Survived']

# On sépare les données, pour créer et tester le modèle (on ne peut pas faire les deux sur les mêmes données, car les estimateurs n'auraient pas de validité externe)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Spécifions le modèle
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

# Créons le vecteur des valeurs prédites
y_pred = model.predict(X_test)

# Calculons la précision du modèle.
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
#On observe qu'en chosissant les variables explicatives selon le critère de corrélation, on passe d'un pouvoir prédictif de 0.69 à 0.80.

In [ ]:
#Maintenant, utilisons un autre critère de choix.
#Ici, nous intégrons les variables dans le modèle selon que leur association avec la survie est statistiquement significative au seuil de 5%.

In [ ]:

# On extrait les variables numériques
numeric_cols = df.select_dtypes(include=[np.number])  

# Ce vecteur vide contiendra les variables significatives après
significant_features = []

# Pour tester les différents variables on utilise une boucle
for col in numeric_cols.columns:
    if col != 'Survived':  # On ne teste pas la variable expliquée
        survived_data = df[df['Survived'] == 1][col]
        not_survived_data = df[df['Survived'] == 0][col]

        # Faisons les t-test
        t_stat, p_value = stats.ttest_ind(survived_data, not_survived_data)

        # On affiche les résultats des t-test
        print(f"T-statistic for '{col}': {t_stat:.4f}")   
        print(f"p-value for '{col}': {p_value:.4f}")      
        print("-------------------------")

        # On garde les variables selon le seuil de significativité choisi (5%)
        if p_value < 0.05:
            significant_features.append(col)

# On affiche les variables statistiquement significatives 
print("Variables à p-value < 0.05:")
print(*significant_features, sep='\n')  # Affichons chaque variable l'une après l'autre